In [1]:
import pandas as pd
import os
from collections import Counter

取得庫存所有資料

In [4]:
# 月份
months = [1,2,3,4,5,6,7,8,9,10,11,12]
# 年分
years = [108,109,110]
# 病房
# rooms = ['五病房','六病房','加護病房','呼吸照護病房']
rooms = ['六病房']
final_data = pd.DataFrame()

for year in years:
    for room in rooms:
        total_data =  pd.DataFrame()
        for month in months:
            #暫存每個月分的領用量
            usage_temp = pd.DataFrame()
        #     每個月分的所有資料
            temp =  pd.DataFrame()
            # sheet_name控制開起的頁面

    #         temp=pd.read_excel(f"../成大醫院物料預測/2019~2021庫存/{year}年度病房衛材帳務日期/{year}年衛材批價明細(帳務日期)-{room}.xlsx",sheet_name = f'{month}月',skiprows=2,index_col=0)
            temp=pd.read_excel(f"../成大醫院物料預測/2019~2021庫存/{year}年度病房出入庫日報表/{year}年度出入庫日報表-{room}.xlsx",sheet_name = f'{month}月',index_col=0)
        #         total_data=pd.read_excel(f"../成大醫院物料預測/2019~2021庫存/{year}年度病房衛材帳務日期/{year}年衛材批價明細(帳務日期)-{room}.xlsx",sheet_name = f'{month}月',index_col=0)
            temp = temp[temp.columns.drop(list(temp.filter(regex='Unnamed')))]
            total_data = pd.concat([total_data, temp])

        # 只刪除日期是空值的
        print(total_data.index)
        total_data.reset_index(inplace=True)
        total_data = total_data.rename(columns = {'index':'日期'})
        total_data = total_data.dropna(subset=["日期"])



        # 加入週的欄位
        total_data["日期"] = pd.to_datetime(total_data["日期"] ,format='%Y/%m/%d')
        total_data['週'] = total_data['日期'].dt.isocalendar().week
    final_data =  pd.concat([final_data, total_data])
# 全部資料合併
#     total_data.to_csv(f"./全部資料合併/{year+1911}_{room}_合併檔.csv",encoding='utf_8_sig') 

# 讀取所有料號，並刪除相同的料號的值
final_data = final_data.drop_duplicates(subset=['料號'])
final_data= final_data[['料號']]
final_data['料號'] = final_data['料號'].str.strip()
final_data

Index(['2019/01/29',          nan, '2019/01/03', '2019/01/08', '2019/01/15',
       '2019/01/22', '2019/01/29',          nan, '2019/01/02', '2019/01/08',
       ...
       '2019/12/20', '2019/12/23', '2019/12/24', '2019/12/31',          nan,
       '2019/12/03',          nan, '2019/12/03', '2019/12/17', '2019/12/24'],
      dtype='object', name='日期', length=7219)
Index(['2020/01/10',          nan, '2020/01/07', '2020/01/14', '2020/01/16',
       '2020/01/18',          nan, '2020/01/16',          nan, '2020/01/18',
       ...
                nan, '2020/12/03', '2020/12/04', '2020/12/09', '2020/12/10',
       '2020/12/11', '2020/12/15', '2020/12/26', '2020/12/29', '2020/12/31'],
      dtype='object', name='日期', length=7987)
Index(['2021/01/12',          nan, '2021/01/25', '2021/01/26', '2021/01/28',
       '2021/01/29',          nan, '2021/01/04', '2021/01/12', '2021/01/23',
       ...
       '2021/12/07', '2021/12/16', '2021/12/28',          nan, '2021/12/28',
                nan, '2021

,料號
0,A00120017
2,A00120033
8,A00120067
12,A00120102
14,A00120110
...,...
2548,M00030327
3635,B13700009
3960,V02204150
6398,V01100853


取得五病房有紀錄所有料號

In [ ]:
record_data =  pd.DataFrame()
for year in years:
    temp =  pd.DataFrame()
    temp=pd.read_csv(f"../以月為單位/{year+1911}_{room}_衛材使用量.csv")
    record_data = pd.concat([record_data, temp])

# 讀取所有料號，並刪除相同的料號的值
record_data = record_data.drop_duplicates(subset=['料號'])
record_data = record_data[['料號']]
record_data['料號'] = record_data['料號'].str.strip()
record_data

In [ ]:
final_data=final_data.append(record_data).reset_index(drop=True)
final_data=final_data.drop_duplicates(subset=['料號'],keep=False)

final_data.to_csv(f"./{room}沒有紀錄衛材總類.csv",encoding='utf_8_sig') 




和全部資料合併，查看找出的93筆資料的品名

In [ ]:
all_Eisai = pd.read_excel(f"../../大庫所有衛材品項及實際撥補量資料.xlsx",sheet_name = "大庫房的所有衛材品項資料",index_col=0)
find_name = pd.read_csv(f"./{room}沒有紀錄衛材總類.csv")
find_name = find_name[find_name.columns.drop(list(find_name.filter(regex='Unnamed')))]
all_Eisai = all_Eisai[all_Eisai.columns.drop(list(all_Eisai.filter(regex='Unnamed')))]
all_Eisai.reset_index(inplace=True)
all_Eisai = all_Eisai.rename(columns = {'Order_Code':'料號'})
all_Eisai['料號'] = all_Eisai['料號'].str.strip()
data = pd.merge(find_name,all_Eisai, on=["料號"],how="inner")

data
data.to_csv(f"./{room}最終需要計算衛材總類.csv",encoding='utf_8_sig') 


